In [1]:
# Ячейка 1: Импорты и настройка
import os
import pandas as pd
import polars as pl
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.feature_selection import VarianceThreshold
import warnings
import gc
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import spearmanr
import joblib

warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")
print(f"Доступно GPU: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Название GPU: {torch.cuda.get_device_name(0)}")

NUM_TARGET_COLUMNS = 424

# Глобальные переменные
models = []
scaler = None
feature_selector = None
feature_cols = None
base_cols = None
is_initialized = False
model_val_scores = []
ensemble_weights_global = None

Используется устройство: cuda
Доступно GPU: 1
Название GPU: NVIDIA L4


In [2]:
class TimeSeriesAugmentation:
    """Продвинутая аугментация для финансовых временных рядов"""
    
    @staticmethod
    def add_gaussian_noise(features, noise_level=0.01):
        """Добавление гауссовского шума"""
        noise = torch.randn_like(features) * noise_level
        return features + noise
    
    @staticmethod
    def time_warp(features, warp_factor=0.1):
        """Временное искажение - сдвиг признаков во времени"""
        batch_size, seq_len = features.shape
        warp_strength = torch.randn(batch_size, 1, device=features.device) * warp_factor
        warped_indices = torch.arange(seq_len, device=features.device).float() + warp_strength
        warped_indices = torch.clamp(warped_indices, 0, seq_len-1)
        
        # Интерполяция
        warped_features = torch.zeros_like(features)
        for i in range(batch_size):
            warped_features[i] = torch.interp(
                torch.arange(seq_len, device=features.device).float(),
                warped_indices[i],
                features[i]
            )
        return warped_features
    
    @staticmethod
    def feature_dropout(features, drop_prob=0.1):
        """Случайное обнуление признаков (как dropout но для фичей)"""
        mask = torch.rand_like(features) > drop_prob
        return features * mask.float()
    
    @staticmethod
    def scaling_augmentation(features, scale_range=(0.9, 1.1)):
        """Случайное масштабирование признаков"""
        scale_factors = torch.rand(features.shape[0], 1, device=features.device) * (scale_range[1] - scale_range[0]) + scale_range[0]
        return features * scale_factors
    
    @staticmethod
    def jittering(features, jitter_strength=0.02):
        """Дрожание - случайные небольшие изменения"""
        jitter = (torch.rand_like(features) - 0.5) * 2 * jitter_strength
        return features + jitter
    
    @staticmethod
    def random_permutation(features, permute_fraction=0.1):
        """Случайная перестановка небольшой части признаков"""
        batch_size, num_features = features.shape
        num_to_permute = int(num_features * permute_fraction)
        
        for i in range(batch_size):
            perm_indices = torch.randperm(num_features)[:num_to_permute]
            features[i, perm_indices] = features[i, perm_indices[torch.randperm(num_to_permute)]]
        
        return features

def advanced_augmentation(features, augmentation_methods=None, augmentation_prob=0.7):
    """Продвинутая аугментация с комбинацией методов"""
    if augmentation_methods is None:
        augmentation_methods = [
            ('noise', 0.3),
            ('scaling', 0.2), 
            ('jitter', 0.2),
            ('dropout', 0.2),
            ('permutation', 0.1)
        ]
    
    augmented_features = features.clone()
    
    # Применяем аугментации с вероятностью
    if torch.rand(1).item() < augmentation_prob:
        for method, prob in augmentation_methods:
            if torch.rand(1).item() < prob:
                if method == 'noise':
                    augmented_features = TimeSeriesAugmentation.add_gaussian_noise(augmented_features, 0.01)
                elif method == 'scaling':
                    augmented_features = TimeSeriesAugmentation.scaling_augmentation(augmented_features)
                elif method == 'jitter':
                    augmented_features = TimeSeriesAugmentation.jittering(augmented_features)
                elif method == 'dropout':
                    augmented_features = TimeSeriesAugmentation.feature_dropout(augmented_features, 0.05)
                elif method == 'permutation':
                    augmented_features = TimeSeriesAugmentation.random_permutation(augmented_features, 0.05)
    
    return augmented_features

In [3]:
# Ячейка 2: Функции потерь
class PearsonCorrelationLoss(nn.Module):
    """Loss функция для оптимизации Pearson Correlation"""
    def __init__(self):
        super().__init__()
    
    def forward(self, pred, target):
        # Для каждого target отдельно
        batch_size = pred.shape[0]
        num_targets = pred.shape[1]
        
        total_loss = 0
        
        for i in range(num_targets):
            pred_col = pred[:, i]
            target_col = target[:, i]
            
            # Центрируем
            pred_centered = pred_col - pred_col.mean()
            target_centered = target_col - target_col.mean()
            
            # Ковариация
            covariance = (pred_centered * target_centered).mean()
            
            # Стандартные отклонения
            pred_std = pred_centered.std()
            target_std = target_centered.std()
            
            # Pearson correlation
            if pred_std > 1e-8 and target_std > 1e-8:
                correlation = covariance / (pred_std * target_std)
                # Минимизируем отрицательную корреляцию
                total_loss += -correlation
            else:
                # Штраф за константные предсказания
                total_loss += 1.0
        
        return total_loss / num_targets

class SpearmanLoss(nn.Module):
    """Approximation of Spearman correlation loss"""
    def __init__(self):
        super().__init__()
    
    def forward(self, pred, target):
        # Используем ранжирование через сортировку
        pred_rank = torch.argsort(torch.argsort(pred, dim=0), dim=0).float()
        target_rank = torch.argsort(torch.argsort(target, dim=0), dim=0).float()
        
        # Нормализуем ранги
        pred_rank = (pred_rank - pred_rank.mean(dim=0)) / (pred_rank.std(dim=0) + 1e-8)
        target_rank = (target_rank - target_rank.mean(dim=0)) / (target_rank.std(dim=0) + 1e-8)
        
        # Correlation (минимизируем отрицательную)
        correlation = (pred_rank * target_rank).mean(dim=0).mean()
        return -correlation

class CombinedCorrelationLoss(nn.Module):
    """Комбинация Pearson + Spearman + MSE"""
    def __init__(self, pearson_weight=0.4, spearman_weight=0.4, mse_weight=0.2):
        super().__init__()
        self.pearson_weight = pearson_weight
        self.spearman_weight = spearman_weight
        self.mse_weight = mse_weight
        
        self.pearson_loss = PearsonCorrelationLoss()
        self.spearman_loss = SpearmanLoss()
        self.mse_loss = nn.MSELoss()
    
    def forward(self, pred, target):
        pearson_l = self.pearson_loss(pred, target)
        spearman_l = self.spearman_loss(pred, target)
        mse_l = self.mse_loss(pred, target)
        
        total_loss = (self.pearson_weight * pearson_l + 
                     self.spearman_weight * spearman_l + 
                     self.mse_weight * mse_l)
        
        return total_loss

In [4]:
# Ячейка 3: Архитектуры моделей
class SimpleEffectiveModel(nn.Module):
    """Простая но эффективная архитектура"""
    def __init__(self, input_size, output_size, hidden_size=512, dropout_rate=0.3):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(hidden_size, hidden_size // 2),
            nn.BatchNorm1d(hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.8),
            
            nn.Linear(hidden_size // 2, hidden_size // 4),
            nn.BatchNorm1d(hidden_size // 4),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.6),
            
            nn.Linear(hidden_size // 4, output_size)
        )
        
        # Инициализация весов
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
    
    def forward(self, x):
        return self.net(x)

In [5]:
class SimpleEffectiveModel_Improved(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=512, dropout_rate=0.4):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(hidden_size, hidden_size // 2),
            nn.BatchNorm1d(hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.8),
            
            nn.Linear(hidden_size // 2, hidden_size // 4),
            nn.BatchNorm1d(hidden_size // 4),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.6),
            
            nn.Linear(hidden_size // 4, output_size)
        )
    
    def forward(self, x):
        return self.net(x)

In [6]:

class ResidualModel(nn.Module):
    """Модель с residual connections"""
    def __init__(self, input_size, output_size, hidden_size=256):
        super().__init__()
        
        self.input_proj = nn.Linear(input_size, hidden_size)
        
        self.block1 = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size)
        )
        
        self.block2 = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size)
        )
        
        self.output = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size // 2, output_size)
        )
        
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, x):
        x = self.input_proj(x)
        
        # Residual blocks
        residual = x
        x = self.block1(x)
        x = x + residual
        x = nn.ReLU()(x)
        x = self.dropout(x)
        
        residual = x
        x = self.block2(x)
        x = x + residual
        x = nn.ReLU()(x)
        x = self.dropout(x)
        
        return self.output(x)

class WideModel(nn.Module):
    """Широкая модель с большим количеством нейронов"""
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, output_size)
        )
    
    def forward(self, x):
        return self.net(x)

In [7]:
# Новая: Глубокая модель с skip connections
class DeepSkipModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=512, num_blocks=4):
        super().__init__()
        
        self.input_proj = nn.Linear(input_size, hidden_size)
        
        self.blocks = nn.ModuleList()
        for i in range(num_blocks):
            self.blocks.append(nn.Sequential(
                nn.Linear(hidden_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(hidden_size, hidden_size),
                nn.BatchNorm1d(hidden_size)
            ))
        
        self.output = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.input_proj(x)
        
        for block in self.blocks:
            residual = x
            x = block(x)
            x = x + residual
            x = nn.ReLU()(x)
            x = self.dropout(x)
        
        return self.output(x)

In [8]:
class AttentionModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=512, num_heads=8):
        super().__init__()
        
        self.input_proj = nn.Linear(input_size, hidden_size)
        
        # Self-attention
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=num_heads,
            dropout=0.1,
            batch_first=True
        )
        
        self.ffn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size * 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size * 2, hidden_size)
        )
        
        self.norm1 = nn.LayerNorm(hidden_size)
        self.norm2 = nn.LayerNorm(hidden_size)
        self.output = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.input_proj(x)
        
        # Add sequence dimension for attention
        x_seq = x.unsqueeze(1)
        
        # Self-attention with residual
        attn_out, _ = self.attention(x_seq, x_seq, x_seq)
        x = x + attn_out.squeeze(1)
        x = self.norm1(x)
        
        # Feed-forward with residual
        ffn_out = self.ffn(x)
        x = x + ffn_out
        x = self.norm2(x)
        
        return self.output(x)

In [9]:
class EnsembleInsideModel(nn.Module):
    def __init__(self, input_size, output_size, num_experts=3):
        super().__init__()
        
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_size, 384),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(384, 256)
            ) for _ in range(num_experts)
        ])
        
        self.gate = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, num_experts),
            nn.Softmax(dim=1)
        )
        
        self.output = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, output_size)
        )
    
    def forward(self, x):
        gates = self.gate(x)
        expert_outputs = [expert(x) for expert in self.experts]
        
        # Weighted combination
        combined = torch.zeros_like(expert_outputs[0])
        for i, expert_out in enumerate(expert_outputs):
            combined += gates[:, i:i+1] * expert_out
        
        return self.output(combined)

In [10]:
# Ячейка 4: Создание признаков
def create_optimized_features(df, base_cols_ref=None, max_features=800):
    """Создает оптимизированный набор признаков"""
    features = df.copy()
    
    if base_cols_ref is not None:
        numeric_cols = [c for c in base_cols_ref if c in df.columns]
    else:
        numeric_cols = [c for c in df.columns 
                       if c not in ['date_id', 'is_scored'] 
                       and pd.api.types.is_numeric_dtype(df[c])]
    
    # Ограничиваем количество базовых колонок
    numeric_cols = numeric_cols[:min(100, len(numeric_cols))]
    
    print(f"Обрабатывается {len(numeric_cols)} базовых колонок...")
    
    feature_count = 0
    for col in numeric_cols:
        try:
            with np.errstate(divide='ignore', invalid='ignore'):
                # Базовые returns
                features[f'{col}_return_1d'] = df[col].pct_change(1)
                features[f'{col}_return_5d'] = df[col].pct_change(5)
                
                # Скользящие статистики
                for window in [5, 10, 20]:
                    features[f'{col}_ma_{window}'] = df[col].rolling(window, min_periods=1).mean()
                    features[f'{col}_std_{window}'] = df[col].rolling(window, min_periods=1).std()
                
                # Z-score
                rolling_mean = df[col].rolling(20, min_periods=1).mean()
                rolling_std = df[col].rolling(20, min_periods=1).std()
                features[f'{col}_zscore'] = (df[col] - rolling_mean) / (rolling_std + 1e-8)
                
                # Lag features
                for lag in [1, 2, 3]:
                    features[f'{col}_lag_{lag}'] = df[col].shift(lag)
                
                # Volatility
                features[f'{col}_volatility_20'] = df[col].rolling(20).std() / (df[col].rolling(20).mean() + 1e-8)
                
                feature_count += 11  # 11 features per column
                
                # Останавливаемся если достигли лимита
                if feature_count >= max_features:
                    print(f"Достигнут лимит в {max_features} признаков")
                    break
                    
        except Exception as e:
            continue
    
    # Заполняем NaN
    features = features.fillna(0)
    
    # Удаляем колонки с нулевой дисперсией
    selector = VarianceThreshold()
    non_constant_features = selector.fit(features.select_dtypes(include=[np.number])).get_support()
    feature_columns = features.select_dtypes(include=[np.number]).columns[non_constant_features]
    features = features[feature_columns]
    
    print(f"Создано {len(features.columns)} признаков после фильтрации")
    
    return features

In [11]:
def select_best_features_vectorized_fixed(features, targets, n_features=500):
    """Исправленная векторизованная версия с обработкой NaN"""
    import torch
    
    print("Векторизованный отбор признаков с GPU...")
    
    # Фильтруем колонки
    feature_cols = [c for c in features.columns if c not in ['date_id', 'is_scored']]
    features_filtered = features[feature_cols]
    
    # Перенос на GPU
    X = torch.tensor(features_filtered.values, dtype=torch.float32, device=device)
    y = torch.tensor(targets.values, dtype=torch.float32, device=device)
    
    # Заменяем NaN и Inf на 0
    X = torch.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    y = torch.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Центрируем данные
    X_centered = X - X.mean(dim=0)
    y_centered = y - y.mean(dim=0)
    
    # Вычисляем ковариационную матрицу
    covariance = torch.mm(X_centered.T, y_centered) / (X.shape[0] - 1)
    
    # Вычисляем стандартные отклонения с защитой от деления на 0
    x_std = X_centered.std(dim=0, unbiased=True)
    y_std = y_centered.std(dim=0, unbiased=True)
    
    # Защита от нулевых стандартных отклонений
    x_std = torch.where(x_std < 1e-8, torch.ones_like(x_std), x_std)
    y_std = torch.where(y_std < 1e-8, torch.ones_like(y_std), y_std)
    
    # Вычисляем матрицу корреляций Пирсона
    correlation_matrix = covariance / (x_std.unsqueeze(1) * y_std.unsqueeze(0))
    
    # Заменяем NaN в корреляциях на 0
    correlation_matrix = torch.nan_to_num(correlation_matrix, nan=0.0)
    
    # Усредняем абсолютные корреляции по targets
    feature_importances = correlation_matrix.abs().mean(dim=1)
    
    # Сортируем признаки по важности
    sorted_indices = torch.argsort(feature_importances, descending=True)
    selected_features = [feature_cols[i] for i in sorted_indices[:n_features].cpu().numpy()]
    
    print(f"Отобрано {len(selected_features)} лучших признаков")
    if len(feature_importances) > 0:
        best_score = feature_importances[sorted_indices[0]].item()
        print(f"Лучший признак: {selected_features[0]} (корреляция: {best_score:.4f})")
    
    # Очистка памяти
    del X, y, X_centered, y_centered, covariance, correlation_matrix
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return selected_features

In [12]:
# Ячейка 5: Функции обучения
def calculate_spearman_score(predictions, targets):
    """Вычисляет средний Spearman correlation"""
    correlations = []
    
    for i in range(targets.shape[1]):
        pred_col = predictions[:, i]
        target_col = targets[:, i]
        
        if len(np.unique(pred_col)) > 1 and len(np.unique(target_col)) > 1:
            try:
                corr, _ = spearmanr(pred_col, target_col)
                if not np.isnan(corr):
                    correlations.append(corr)
            except:
                continue
    
    return np.mean(correlations) if correlations else 0.0

In [13]:
def augment_features(features, noise_level=0.01):
    """Добавляет шум к признакам для аугментации"""
    noise = torch.randn_like(features) * noise_level
    return features + noise

In [14]:
def train_model_with_augmentation(model, X_train_t, y_train_t, X_val_t, y_val_t, 
                                epochs=500, patience=50, lr=0.001, model_name="", 
                                use_augmentation=True, augmentation_strength=0.7):
    """Функция обучения с продвинутой аугментацией"""
    
    criterion = CombinedCorrelationLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.05)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=8, factor=0.5)
    
    best_val_score = -float('inf')
    best_model_state = None
    epochs_no_improve = 0
    
    print(f"Начало обучения {model_name} с аугментацией: {use_augmentation}")
    
    model.train()
    
    for epoch in range(epochs):
        if epochs_no_improve >= patience:
            print(f"Early stopping на эпохе {epoch+1}")
            break
        
        # Обучение с аугментацией
        optimizer.zero_grad()
        
        if use_augmentation:
            # Применяем аугментацию только к тренировочным данным
            augmented_features = advanced_augmentation(X_train_t, augmentation_prob=augmentation_strength)
            train_outputs = model(augmented_features)
        else:
            train_outputs = model(X_train_t)
            
        train_loss = criterion(train_outputs, y_train_t)
        train_loss.backward()
        
        # Gradient clipping для стабильности
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        # Валидация (БЕЗ аугментации)
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_t)
            val_score = calculate_spearman_score(val_outputs.cpu().numpy(), y_val_t.cpu().numpy())
            
            train_preds = train_outputs.detach().cpu().numpy()
            train_true = y_train_t.cpu().numpy()
            train_score = calculate_spearman_score(train_preds, train_true)
        
        # Обновление лучших результатов
        if val_score > best_val_score:
            best_val_score = val_score
            best_model_state = model.state_dict().copy()
            epochs_no_improve = 0
            improvement_msg = "✓ УЛУЧШЕНИЕ"
        else:
            epochs_no_improve += 1
            improvement_msg = f"NO IMPROVE ({epochs_no_improve}/{patience})"
        
        model.train()
        scheduler.step(train_loss)
        
        # Логирование
        if (epoch + 1) % 10 == 0 or epoch == 0 or epochs_no_improve >= patience:
            current_lr = optimizer.param_groups[0]['lr']
            aug_status = "AUG" if use_augmentation else "NO_AUG"
            print(f"{model_name} | {aug_status} | Ep {epoch+1:3d}/{epochs} | "
                  f"TrL: {train_loss.item():.6f} | "
                  f"TrScore: {train_score:.4f} | "
                  f"ValScore: {val_score:.4f} | "
                  f"LR: {current_lr:.6f} | {improvement_msg}")
    
    # Загрузка лучших весов
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    model.eval()
    
    return model, best_val_score

In [15]:
def initialize_models_with_augmentation():
    global models, scaler, feature_cols, base_cols, model_val_scores, is_initialized, device
    
    if is_initialized:
        return
    
    print("="*70)
    print(f"ИНИЦИАЛИЗАЦИЯ С АУГМЕНТАЦИЕЙ НА {device}")
    print("="*70)
    
    # Загрузка данных
    train = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/train.csv')
    train_labels = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/train_labels.csv')
    
    # Базовые колонки
    base_cols = [c for c in train.columns 
                 if c not in ['date_id'] 
                 and pd.api.types.is_numeric_dtype(train[c])]
    
    print(f"Базовых колонок: {len(base_cols)}")
    print("Создание оптимизированных признаков...")
    
    # Создание признаков
    train_features = create_optimized_features(train, base_cols_ref=base_cols, max_features=800)
    
    # Выбор лучших признаков
    target_cols = [f'target_{i}' for i in range(424)]
    selected_features = select_best_features_vectorized_fixed(train_features, train_labels[target_cols], n_features=500)
    
    feature_cols = selected_features
    
    # Подготовка данных
    X_train = train_features[feature_cols].fillna(0).values
    y_train = train_labels[target_cols].fillna(0).values
    
    X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
    y_train = np.nan_to_num(y_train, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Масштабирование
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Перенос на GPU
    X_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_tensor = torch.FloatTensor(y_train).to(device)
    
    # Разделение на train/validation
    split_idx = int(len(X_train_scaled) * 0.85)
    X_train_t, X_val_t = X_tensor[:split_idx], X_tensor[split_idx:]
    y_train_t, y_val_t = y_tensor[:split_idx], y_tensor[split_idx:]
    
    print(f"Train: {len(X_train_t)}, Validation: {len(X_val_t)}")
    
    # Конфигурация моделей с разной силой аугментации
    model_configs = [
        (SimpleEffectiveModel_Improved, "SimpleEffective_Improved", 0.001, True, 0.7),
        (ResidualModel, "Residual", 0.001, True, 0.8),
        (WideModel, "Wide", 0.0005, True, 0.6),
        (DeepSkipModel, "DeepSkip", 0.001, True, 0.7),
        (AttentionModel, "Attention", 0.0008, True, 0.5),  # Меньше аугментации для attention
        (EnsembleInsideModel, "EnsembleInside", 0.001, True, 0.7),
    ]
    
    # Обучение всех моделей с аугментацией
    for i, (ModelClass, name, lr, use_aug, aug_strength) in enumerate(model_configs):
        print(f"\n{'='*70}")
        print(f"МОДЕЛЬ {i+1}/{len(model_configs)}: {name} (Аугментация: {use_aug})")
        print(f"{'='*70}")
        
        try:
            # Создание модели
            if name == "Wide":
                model = ModelClass(X_train_scaled.shape[1], 424).to(device)
            elif name == "Attention":
                model = ModelClass(X_train_scaled.shape[1], 424, hidden_size=384).to(device)
            elif name == "DeepSkip":
                model = ModelClass(X_train_scaled.shape[1], 424, hidden_size=512, num_blocks=5).to(device)
            elif name == "EnsembleInside":
                model = ModelClass(X_train_scaled.shape[1], 424, num_experts=4).to(device)
            else:
                model = ModelClass(X_train_scaled.shape[1], 424, hidden_size=512).to(device)
            
            # Обучение с аугментацией
            model, val_score = train_model_with_augmentation(
                model, X_train_t, y_train_t, X_val_t, y_val_t, 
                epochs=200, patience=20, lr=lr, model_name=name,
                use_augmentation=use_aug, augmentation_strength=aug_strength
            )
            
            models.append(model)
            model_val_scores.append(val_score)
            
            print(f"✅ {name} завершена. Best Val Score: {val_score:.4f}")
            
        except Exception as e:
            print(f"❌ ОШИБКА при обучении модели {name}: {e}")
            import traceback
            traceback.print_exc()
            continue
        
        # Очистка GPU памяти
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Проверка что хоть что-то обучилось
    if not models:
        raise Exception("❌ НИ ОДНА МОДЕЛЬ НЕ БЫЛА УСПЕШНО ОБУЧЕНА!")
    
    # Вычисление весов для ансамбля
    weights = np.array(model_val_scores)
    weights = weights / weights.sum()
    
    print(f"\n{'='*70}")
    print(f"ВЕСА МОДЕЛЕЙ С АУГМЕНТАЦИЕЙ:")
    for i, (config, weight, val_score) in enumerate(zip(model_configs, weights, model_val_scores)):
        if i < len(models):
            aug_status = "AUG" if config[3] else "NO_AUG"
            print(f"  Модель {i+1} ({config[1]}): вес={weight:.4f}, val_score={val_score:.4f} [{aug_status}]")
    print(f"{'='*70}\n")
    
    is_initialized = True
    
    # Сохранение scaler и feature_cols
    joblib.dump(scaler, 'scaler.joblib')
    joblib.dump(feature_cols, 'feature_cols.joblib')
    print("Scaler и feature_cols сохранены")

In [16]:
def prepare_features(df):
    global feature_cols, scaler
    
    # Создаем фичи тем же способом что и при обучении
    test_features = create_optimized_features(df, base_cols_ref=base_cols, max_features=800)
    
    # Используем только те фичи, которые были отобраны при обучении
    available_features = [f for f in feature_cols if f in test_features.columns]
    
    X_test = np.zeros((len(df), len(feature_cols)))
    
    for i, col in enumerate(feature_cols):
        if col in test_features.columns:
            X_test[:, i] = test_features[col].fillna(0).values
    
    X_test = np.nan_to_num(X_test, nan=0.0, posinf=0.0, neginf=0.0)
    
    return X_test

In [17]:
# Ячейка 7: Функции предсказания
def predict(test, label_lags_1_batch, label_lags_2_batch, label_lags_3_batch, label_lags_4_batch):
    global models, scaler, feature_cols, model_val_scores, is_initialized, device
    
    if not is_initialized:
        initialize_models_fixed()
    
    try:
        test_pd = test.to_pandas()
        X_test = prepare_features(test_pd)
        X_test_scaled = scaler.transform(X_test[-1:])
        
        X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
        
        # Взвешенный ансамбль
        all_preds = []
        with torch.no_grad():
            for model in models:
                pred = model(X_test_tensor)
                pred_cpu = pred.cpu().numpy()[0]
                all_preds.append(pred_cpu)
        
        # Вычисляем веса (пропорционально val score)
        weights = np.array(model_val_scores)
        weights = weights / weights.sum()
        
        # Взвешенное усреднение
        predictions = np.average(all_preds, axis=0, weights=weights)
        
        # Мягкое ограничение значений
        predictions = np.tanh(predictions * 10) * 0.1  # Мягкое ограничение до [-0.1, 0.1]
        predictions = np.nan_to_num(predictions, nan=0.0, posinf=0.0, neginf=0.0)
        
        return pl.DataFrame({f'target_{i}': [float(predictions[i])] for i in range(NUM_TARGET_COLUMNS)})
        
    except Exception as e:
        print(f"Ошибка predict: {e}")
        return pl.DataFrame({f'target_{i}': [0.0] for i in range(NUM_TARGET_COLUMNS)})

@torch.no_grad()
def efficient_predict_batch(test_data, models, weights=None, batch_size=1024):
    """Эффективное предсказание с батчингом"""
    if weights is None:
        weights = np.ones(len(models)) / len(models)
    
    # Если test_data уже pandas DataFrame, используем как есть
    if hasattr(test_data, 'to_pandas'):
        test_pd = test_data.to_pandas()
    else:
        test_pd = test_data
    
    X_test = prepare_features(test_pd)
    X_test_scaled = scaler.transform(X_test)
    
    # Батчинг для больших данных
    all_predictions = []
    
    for i in range(0, len(X_test_scaled), batch_size):
        batch = X_test_scaled[i:i+batch_size]
        X_batch_tensor = torch.FloatTensor(batch).to(device)
        
        batch_preds = []
        for model, weight in zip(models, weights):
            pred = model(X_batch_tensor).cpu().numpy()
            batch_preds.append(pred * weight)
        
        batch_ensemble = np.sum(batch_preds, axis=0)
        
        # Мягкое ограничение
        batch_ensemble = np.tanh(batch_ensemble * 10) * 0.1
        all_predictions.append(batch_ensemble)
    
    predictions = np.vstack(all_predictions)
    predictions = np.nan_to_num(predictions, nan=0.0, posinf=0.0, neginf=0.0)
    
    return predictions

def create_submission_file():
    global models, scaler, feature_cols, is_initialized, device, model_val_scores
    
    print("\nСоздание submission.parquet...")
    
    if not is_initialized:
        raise Exception("Модели не инициализированы!")
    
    test = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/test.csv')
    
    print("Генерация предсказаний...")
    
    # Вычисляем веса
    weights = np.array(model_val_scores)
    weights = weights / weights.sum()
    
    # Используем батчинговое предсказание
    predictions = efficient_predict_batch(test, models, weights)
    
    submission = pd.DataFrame({'date_id': test['date_id'].values})
    for i in range(424):
        submission[f'target_{i}'] = predictions[:, i]
    
    if 'is_scored' in test.columns:
        submission = submission[test['is_scored'] == True].reset_index(drop=True)
    
    submission = submission.fillna(0).replace([np.inf, -np.inf], 0)
    submission.to_parquet('submission.parquet', index=False, engine='pyarrow')
    
    print(f"Готово: {submission.shape}")
    
    # Очищаем GPU память
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

In [18]:
# Ячейка 8: Оценка модели
def calculate_kaggle_score(predictions, targets):
    """
    ПРАВИЛЬНАЯ метрика соревнования: Modified Sharpe Ratio
    Score = (Mean Spearman Correlation / Std Spearman Correlation) * 100,000
    """
    
    print("\n" + "="*70)
    print("ВЫЧИСЛЕНИЕ KAGGLE SCORE (Modified Sharpe Ratio)")
    print("="*70)
    
    correlations = []
    failed_targets = 0
    
    # Для каждого target вычисляем Spearman correlation
    for i in range(targets.shape[1]):
        pred_col = predictions[:, i]
        true_col = targets[:, i]
        
        # Проверяем что есть вариация в данных
        if len(np.unique(pred_col)) < 2 or len(np.unique(true_col)) < 2:
            failed_targets += 1
            continue
        
        # Spearman rank correlation
        try:
            corr, p_value = spearmanr(pred_col, true_col)
            if not np.isnan(corr) and not np.isinf(corr):
                correlations.append(corr)
            else:
                failed_targets += 1
        except Exception as e:
            failed_targets += 1
            continue
    
    if len(correlations) == 0:
        print("\n⚠️ Не удалось вычислить корреляции!")
        return {
            'kaggle_score': 0.0,
            'sharpe_ratio': 0.0,
            'mean_correlation': 0.0,
            'std_correlation': 0.0,
            'correlations': np.array([])
        }
    
    correlations = np.array(correlations)
    
    # Статистика
    mean_corr = np.mean(correlations)
    std_corr = np.std(correlations)
    
    # Kaggle Score (масштабированный Modified Sharpe Ratio)
    if std_corr > 1e-8:
        sharpe_ratio = mean_corr / std_corr
        kaggle_score = sharpe_ratio * 100000
    else:
        sharpe_ratio = 0
        kaggle_score = 0
    
    print(f"\nСтатистика Spearman Correlations:")
    print(f"  Успешно: {len(correlations)}/{targets.shape[1]} targets")
    print(f"  Mean:    {mean_corr:.6f}")
    print(f"  Std:     {std_corr:.6f}")
    print(f"  Min:     {np.min(correlations):.6f}")
    print(f"  Max:     {np.max(correlations):.6f}")
    
    positive = (correlations > 0).sum()
    negative = (correlations < 0).sum()
    
    print(f"\n  Positive correlations: {positive}/{len(correlations)} ({100*positive/len(correlations):.1f}%)")
    print(f"  Negative correlations: {negative}/{len(correlations)} ({100*negative/len(correlations):.1f}%)")
    
    print(f"\n{'='*70}")
    print(f"KAGGLE SCORE (Modified Sharpe Ratio):")
    print(f"{'='*70}")
    print(f"  Sharpe Ratio:        {sharpe_ratio:.6f}")
    print(f"  KAGGLE SCORE:        {kaggle_score:.2f}")
    
    # Интерпретация
    if kaggle_score > 100000:
        print(f"\n  ✅ ОТЛИЧНЫЙ результат! (> 100,000)")
    elif kaggle_score > 50000:
        print(f"\n  ✓ Хороший результат (> 50,000)")
    elif kaggle_score > 0:
        print(f"\n  ⚠ Слабый результат (> 0, но < 50,000)")
    else:
        print(f"\n  ❌ ПЛОХОЙ результат (отрицательный score)")
    
    print(f"{'='*70}\n")
    
    return {
        'kaggle_score': kaggle_score,
        'sharpe_ratio': sharpe_ratio,
        'mean_correlation': mean_corr,
        'std_correlation': std_corr,
        'correlations': correlations
    }

def full_evaluation():
    """
    Полная оценка модели
    """
    global models, scaler, feature_cols, base_cols, model_val_scores, is_initialized, device
    
    if not is_initialized:
        print("Модели не инициализированы!")
        return
    
    print("\n" + "🎯"*35)
    print("ПОЛНАЯ ОЦЕНКА МОДЕЛИ")
    print("🎯"*35)
    
    # Загружаем данные
    train = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/train.csv')
    train_labels = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/train_labels.csv')
    
    # Создаем фичи
    train_features = create_optimized_features(train, base_cols_ref=base_cols, max_features=800)
    
    target_cols = [f'target_{i}' for i in range(424)]
    
    # Используем только отобранные фичи
    available_features = [f for f in feature_cols if f in train_features.columns]
    X_train = train_features[available_features].fillna(0).values
    y_train = train_labels[target_cols].fillna(0).values
    
    X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
    y_train = np.nan_to_num(y_train, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Validation set (последние 15%)
    split_idx = int(len(X_train) * 0.85)
    X_val = X_train[split_idx:]
    y_val = y_train[split_idx:]
    
    print(f"\n1️⃣  VALIDATION SCORE:")
    print(f"Validation set size: {len(X_val)} samples")
    
    # Предсказания
    X_val_scaled = scaler.transform(X_val)
    X_val_tensor = torch.FloatTensor(X_val_scaled).to(device)
    
    all_preds = []
    with torch.no_grad():
        for model in models:
            model.eval()
            pred = model(X_val_tensor)
            pred_cpu = pred.cpu().numpy()
            all_preds.append(pred_cpu)
    
    # Взвешенный ансамбль
    weights = np.array(model_val_scores)
    weights = weights / weights.sum()
    
    val_preds = np.average(all_preds, axis=0, weights=weights)
    val_preds = np.tanh(val_preds * 10) * 0.1
    val_preds = np.nan_to_num(val_preds, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Вычисляем Kaggle Score
    val_results = calculate_kaggle_score(val_preds, y_val)
    
    # Test set (последние 90 дней)
    print(f"\n2️⃣  TEST SCORE:")
    test_train = train.tail(90).reset_index(drop=True)
    test_labels = train_labels.tail(90).reset_index(drop=True)
    
    X_test = prepare_features(test_train)
    y_test = test_labels[target_cols].fillna(0).values
    y_test = np.nan_to_num(y_test, nan=0.0, posinf=0.0, neginf=0.0)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    
    all_preds = []
    with torch.no_grad():
        for model in models:
            model.eval()
            pred = model(X_test_tensor)
            pred_cpu = pred.cpu().numpy()
            all_preds.append(pred_cpu)
    
    test_preds = np.average(all_preds, axis=0, weights=weights)
    test_preds = np.tanh(test_preds * 10) * 0.1
    test_preds = np.nan_to_num(test_preds, nan=0.0, posinf=0.0, neginf=0.0)
    
    test_results = calculate_kaggle_score(test_preds, y_test)
    
    # Сравнение
    print("\n" + "="*70)
    print("📊 СРАВНЕНИЕ РЕЗУЛЬТАТОВ:")
    print("="*70)
    print(f"{'Метрика':<30} {'Validation':<20} {'Test':<20}")
    print("-"*70)
    print(f"{'KAGGLE SCORE':<30} {val_results['kaggle_score']:<20.2f} {test_results['kaggle_score']:<20.2f}")
    print(f"{'Sharpe Ratio':<30} {val_results['sharpe_ratio']:<20.6f} {test_results['sharpe_ratio']:<20.6f}")
    print(f"{'Mean Correlation':<30} {val_results['mean_correlation']:<20.6f} {test_results['mean_correlation']:<20.6f}")
    print("="*70)
    
    if val_results['kaggle_score'] > 0:
        print("✅ ХОРОШО: Модель показывает положительную корреляцию!")
    else:
        print("❌ ПЛОХО: Модель показывает отрицательную корреляцию!")
    
    print("\n" + "🎯"*35 + "\n")
    
    return {
        'validation': val_results,
        'test': test_results
    }

In [19]:
def main_with_augmentation():
    global is_initialized
    
    print("ЗАПУСК ПАЙПЛАЙНА С ПРОДВИНУТОЙ АУГМЕНТАЦИЕЙ")
    print("="*60)
    
    if not is_initialized:
        print("Начало обучения со сложной аугментацией...")
        initialize_models_with_augmentation()
    
    # Оценка
    print("\n" + "="*60)
    print("ОЦЕНКА МОДЕЛЕЙ С АУГМЕНТАЦИЕЙ")
    print("="*60)
    
    results = full_evaluation()
    
    # Создание submission
    print("\nСоздание submission файла...")
    create_submission_file()
    
    print("\n✅ АУГМЕНТИРОВАННЫЙ АНСАМБЛЬ ГОТОВ!")
    print(f"🏆 Ваш ожидаемый Kaggle Score: {results['validation']['kaggle_score']:.2f}")
    print(f"📊 Sharpe Ratio: {results['validation']['sharpe_ratio']:.6f}")
    print(f"🔢 Количество моделей: {len(models)}")
    print(f"🎯 Использована продвинутая аугментация")
    print(f"📁 Submission файл: submission.parquet")
    print(f"\n🚀 ОТПРАВЛЯЙТЕ НА KAGGLE!")

if __name__ == "__main__":
    main_with_augmentation()

ЗАПУСК ПАЙПЛАЙНА С ПРОДВИНУТОЙ АУГМЕНТАЦИЕЙ
Начало обучения со сложной аугментацией...
ИНИЦИАЛИЗАЦИЯ С АУГМЕНТАЦИЕЙ НА cuda
Базовых колонок: 557
Создание оптимизированных признаков...
Обрабатывается 100 базовых колонок...
Достигнут лимит в 800 признаков
Создано 1507 признаков после фильтрации
Векторизованный отбор признаков с GPU...
Отобрано 500 лучших признаков
Лучший признак: US_Stock_EMB_adj_open_volatility_20 (корреляция: 0.0607)
Train: 1666, Validation: 295

МОДЕЛЬ 1/6: SimpleEffective_Improved (Аугментация: True)
Начало обучения SimpleEffective_Improved с аугментацией: True
SimpleEffective_Improved | AUG | Ep   1/200 | TrL: 0.045389 | TrScore: -0.0007 | ValScore: 0.0014 | LR: 0.001000 | ✓ УЛУЧШЕНИЕ
SimpleEffective_Improved | AUG | Ep  10/200 | TrL: -0.031458 | TrScore: 0.0451 | ValScore: 0.0199 | LR: 0.001000 | ✓ УЛУЧШЕНИЕ
SimpleEffective_Improved | AUG | Ep  20/200 | TrL: -0.114963 | TrScore: 0.1310 | ValScore: 0.0520 | LR: 0.001000 | ✓ УЛУЧШЕНИЕ
SimpleEffective_Improved | AUG |

#### Optuna

In [21]:
# Добавьте эти импорты в начало файла
import optuna
from optuna import Trial
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Расширенный класс для оптимизации ВСЕХ параметров
class ComprehensiveOptunaOptimizer:
    """Комплексная оптимизация ВСЕХ гиперпараметров"""
    
    def __init__(self, X_train_t, y_train_t, X_val_t, y_val_t, feature_cols):
        self.X_train_t = X_train_t
        self.y_train_t = y_train_t
        self.X_val_t = X_val_t
        self.y_val_t = y_val_t
        self.feature_cols = feature_cols
        self.best_params = {}
        self.studies = {}
    
    def objective_comprehensive(self, trial):
        """Комплексная objective функция, оптимизирующая ВСЕ параметры"""
        
        # 1. Выбор архитектуры модели
        model_type = trial.suggest_categorical('model_type', [
            'simple_effective', 'residual', 'wide', 'deep_skip', 
            'attention', 'ensemble_inside'
        ])
        
        # 2. Общие гиперпараметры
        lr = trial.suggest_float('lr', 1e-6, 1e-2, log=True)
        weight_decay = trial.suggest_float('weight_decay', 1e-8, 1e-2, log=True)
        batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256])
        
        # 3. Параметры аугментации
        augmentation_strength = trial.suggest_float('augmentation_strength', 0.1, 0.9)
        use_noise = trial.suggest_categorical('use_noise', [True, False])
        use_scaling = trial.suggest_categorical('use_scaling', [True, False])
        use_jitter = trial.suggest_categorical('use_jitter', [True, False])
        use_dropout_aug = trial.suggest_categorical('use_dropout_aug', [True, False])
        use_permutation = trial.suggest_categorical('use_permutation', [True, False])
        
        # 4. Параметры loss функции
        pearson_weight = trial.suggest_float('pearson_weight', 0.1, 0.8)
        spearman_weight = trial.suggest_float('spearman_weight', 0.1, 0.8)
        mse_weight = trial.suggest_float('mse_weight', 0.05, 0.4)
        
        # Нормализация весов
        total_weight = pearson_weight + spearman_weight + mse_weight
        pearson_weight /= total_weight
        spearman_weight /= total_weight
        mse_weight /= total_weight
        
        # 5. Архитектурные параметры в зависимости от типа модели
        if model_type == 'simple_effective':
            hidden_size = trial.suggest_categorical('hidden_size', [256, 512, 768, 1024, 1536])
            dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.7)
            num_layers = trial.suggest_int('num_layers', 2, 6)
            
            model = self._create_simple_effective_model(
                hidden_size, dropout_rate, num_layers
            )
            
        elif model_type == 'residual':
            hidden_size = trial.suggest_categorical('res_hidden_size', [128, 256, 512, 768])
            num_blocks = trial.suggest_int('num_blocks', 2, 6)
            block_dropout = trial.suggest_float('block_dropout', 0.1, 0.5)
            
            model = self._create_residual_model(
                hidden_size, num_blocks, block_dropout
            )
            
        elif model_type == 'wide':
            layer1_size = trial.suggest_categorical('layer1_size', [512, 768, 1024, 1536, 2048])
            layer2_size = trial.suggest_categorical('layer2_size', [256, 512, 768, 1024])
            layer3_size = trial.suggest_categorical('layer3_size', [128, 256, 512])
            dropout1 = trial.suggest_float('dropout1', 0.2, 0.7)
            dropout2 = trial.suggest_float('dropout2', 0.1, 0.6)
            dropout3 = trial.suggest_float('dropout3', 0.1, 0.5)
            
            model = self._create_wide_model(
                layer1_size, layer2_size, layer3_size, dropout1, dropout2, dropout3
            )
            
        elif model_type == 'deep_skip':
            hidden_size = trial.suggest_categorical('deep_hidden_size', [256, 512, 768, 1024])
            num_blocks = trial.suggest_int('deep_num_blocks', 3, 8)
            block_dropout = trial.suggest_float('deep_dropout', 0.1, 0.5)
            
            model = self._create_deep_skip_model(
                hidden_size, num_blocks, block_dropout
            )
            
        elif model_type == 'attention':
            hidden_size = trial.suggest_categorical('attn_hidden_size', [256, 384, 512, 768])
            num_heads = trial.suggest_categorical('num_heads', [4, 8, 16])
            ffn_multiplier = trial.suggest_float('ffn_multiplier', 1.0, 4.0)
            attn_dropout = trial.suggest_float('attn_dropout', 0.05, 0.3)
            
            model = self._create_attention_model(
                hidden_size, num_heads, ffn_multiplier, attn_dropout
            )
            
        elif model_type == 'ensemble_inside':
            num_experts = trial.suggest_int('num_experts', 2, 6)
            expert_size = trial.suggest_categorical('expert_size', [128, 256, 384, 512])
            gate_size = trial.suggest_categorical('gate_size', [64, 128, 256])
            expert_dropout = trial.suggest_float('expert_dropout', 0.1, 0.5)
            
            model = self._create_ensemble_model(
                num_experts, expert_size, gate_size, expert_dropout
            )
        
        # 6. Параметры оптимизатора
        optimizer_type = trial.suggest_categorical('optimizer', ['adam', 'adamw', 'sgd', 'rmsprop'])
        use_scheduler = trial.suggest_categorical('use_scheduler', [True, False])
        grad_clip = trial.suggest_float('grad_clip', 0.1, 2.0)
        
        # Создание оптимизатора
        if optimizer_type == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        elif optimizer_type == 'adamw':
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        elif optimizer_type == 'sgd':
            optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.9)
        else:  # rmsprop
            optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)
        
        # Scheduler
        if use_scheduler:
            scheduler_type = trial.suggest_categorical('scheduler_type', ['plateau', 'cosine', 'step'])
            if scheduler_type == 'plateau':
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
            elif scheduler_type == 'cosine':
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
            else:  # step
                scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
        else:
            scheduler = None
        
        # 7. Custom loss функция
        criterion = CombinedCorrelationLoss(
            pearson_weight=pearson_weight,
            spearman_weight=spearman_weight,
            mse_weight=mse_weight
        )
        
        # 8. Конфигурация аугментации
        augmentation_methods = []
        if use_noise:
            augmentation_methods.append(('noise', trial.suggest_float('noise_prob', 0.1, 0.8)))
        if use_scaling:
            augmentation_methods.append(('scaling', trial.suggest_float('scaling_prob', 0.1, 0.8)))
        if use_jitter:
            augmentation_methods.append(('jitter', trial.suggest_float('jitter_prob', 0.1, 0.8)))
        if use_dropout_aug:
            augmentation_methods.append(('dropout', trial.suggest_float('dropout_aug_prob', 0.05, 0.4)))
        if use_permutation:
            augmentation_methods.append(('permutation', trial.suggest_float('permutation_prob', 0.05, 0.3)))
        
        # Обучение с комплексными параметрами
        best_val_score = self._train_with_comprehensive_params(
            trial, model, optimizer, scheduler, criterion, 
            augmentation_strength, augmentation_methods, batch_size, grad_clip
        )
        
        return best_val_score
    
    def _create_simple_effective_model(self, hidden_size, dropout_rate, num_layers):
        """Создание SimpleEffectiveModel с оптимизированными параметрами"""
        class DynamicSimpleEffectiveModel(nn.Module):
            def __init__(self, input_size, output_size, hidden_size, dropout_rate, num_layers):
                super().__init__()
                
                layers = []
                current_size = input_size
                
                # Входной слой
                layers.extend([
                    nn.Linear(current_size, hidden_size),
                    nn.BatchNorm1d(hidden_size),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate)
                ])
                current_size = hidden_size
                
                # Скрытые слои
                for i in range(num_layers - 2):
                    next_size = max(hidden_size // (2 ** (i + 1)), 64)
                    layers.extend([
                        nn.Linear(current_size, next_size),
                        nn.BatchNorm1d(next_size),
                        nn.ReLU(),
                        nn.Dropout(dropout_rate * (0.8 ** (i + 1)))
                    ])
                    current_size = next_size
                
                # Выходной слой
                layers.append(nn.Linear(current_size, output_size))
                
                self.net = nn.Sequential(*layers)
            
            def forward(self, x):
                return self.net(x)
        
        return DynamicSimpleEffectiveModel(
            len(self.feature_cols), 424, hidden_size, dropout_rate, num_layers
        ).to(device)
    
    def _create_residual_model(self, hidden_size, num_blocks, block_dropout):
        """Создание ResidualModel с оптимизированными параметрами"""
        class DynamicResidualModel(nn.Module):
            def __init__(self, input_size, output_size, hidden_size, num_blocks, block_dropout):
                super().__init__()
                
                self.input_proj = nn.Linear(input_size, hidden_size)
                self.blocks = nn.ModuleList()
                
                for _ in range(num_blocks):
                    self.blocks.append(nn.Sequential(
                        nn.Linear(hidden_size, hidden_size),
                        nn.BatchNorm1d(hidden_size),
                        nn.ReLU(),
                        nn.Dropout(block_dropout),
                        nn.Linear(hidden_size, hidden_size),
                        nn.BatchNorm1d(hidden_size)
                    ))
                
                self.output = nn.Sequential(
                    nn.Linear(hidden_size, hidden_size // 2),
                    nn.ReLU(),
                    nn.Dropout(block_dropout * 0.5),
                    nn.Linear(hidden_size // 2, output_size)
                )
                self.dropout = nn.Dropout(block_dropout * 0.3)
            
            def forward(self, x):
                x = self.input_proj(x)
                
                for block in self.blocks:
                    residual = x
                    x = block(x)
                    x = x + residual
                    x = nn.ReLU()(x)
                    x = self.dropout(x)
                
                return self.output(x)
        
        return DynamicResidualModel(
            len(self.feature_cols), 424, hidden_size, num_blocks, block_dropout
        ).to(device)
    
    def _create_wide_model(self, layer1_size, layer2_size, layer3_size, dropout1, dropout2, dropout3):
        """Создание WideModel с оптимизированными параметрами"""
        class DynamicWideModel(nn.Module):
            def __init__(self, input_size, output_size, layer1_size, layer2_size, layer3_size, dropout1, dropout2, dropout3):
                super().__init__()
                
                self.net = nn.Sequential(
                    nn.Linear(input_size, layer1_size),
                    nn.BatchNorm1d(layer1_size),
                    nn.ReLU(),
                    nn.Dropout(dropout1),
                    
                    nn.Linear(layer1_size, layer2_size),
                    nn.BatchNorm1d(layer2_size),
                    nn.ReLU(),
                    nn.Dropout(dropout2),
                    
                    nn.Linear(layer2_size, layer3_size),
                    nn.BatchNorm1d(layer3_size),
                    nn.ReLU(),
                    nn.Dropout(dropout3),
                    
                    nn.Linear(layer3_size, output_size)
                )
            
            def forward(self, x):
                return self.net(x)
        
        return DynamicWideModel(
            len(self.feature_cols), 424, layer1_size, layer2_size, layer3_size, dropout1, dropout2, dropout3
        ).to(device)
    
    def _create_deep_skip_model(self, hidden_size, num_blocks, block_dropout):
        """Создание DeepSkipModel с оптимизированными параметрами"""
        class DynamicDeepSkipModel(nn.Module):
            def __init__(self, input_size, output_size, hidden_size, num_blocks, block_dropout):
                super().__init__()
                
                self.input_proj = nn.Linear(input_size, hidden_size)
                self.blocks = nn.ModuleList()
                
                for i in range(num_blocks):
                    self.blocks.append(nn.Sequential(
                        nn.Linear(hidden_size, hidden_size),
                        nn.BatchNorm1d(hidden_size),
                        nn.ReLU(),
                        nn.Dropout(block_dropout),
                        nn.Linear(hidden_size, hidden_size),
                        nn.BatchNorm1d(hidden_size)
                    ))
                
                self.output = nn.Linear(hidden_size, output_size)
                self.dropout = nn.Dropout(block_dropout * 0.7)
            
            def forward(self, x):
                x = self.input_proj(x)
                
                for block in self.blocks:
                    residual = x
                    x = block(x)
                    x = x + residual
                    x = nn.ReLU()(x)
                    x = self.dropout(x)
                
                return self.output(x)
        
        return DynamicDeepSkipModel(
            len(self.feature_cols), 424, hidden_size, num_blocks, block_dropout
        ).to(device)
    
    def _create_attention_model(self, hidden_size, num_heads, ffn_multiplier, attn_dropout):
        """Создание AttentionModel с оптимизированными параметрами"""
        class DynamicAttentionModel(nn.Module):
            def __init__(self, input_size, output_size, hidden_size, num_heads, ffn_multiplier, attn_dropout):
                super().__init__()
                
                self.input_proj = nn.Linear(input_size, hidden_size)
                
                self.attention = nn.MultiheadAttention(
                    embed_dim=hidden_size,
                    num_heads=num_heads,
                    dropout=attn_dropout,
                    batch_first=True
                )
                
                ffn_hidden = int(hidden_size * ffn_multiplier)
                self.ffn = nn.Sequential(
                    nn.Linear(hidden_size, ffn_hidden),
                    nn.ReLU(),
                    nn.Dropout(attn_dropout * 2),
                    nn.Linear(ffn_hidden, hidden_size)
                )
                
                self.norm1 = nn.LayerNorm(hidden_size)
                self.norm2 = nn.LayerNorm(hidden_size)
                self.output = nn.Linear(hidden_size, output_size)
                self.dropout = nn.Dropout(attn_dropout)
            
            def forward(self, x):
                x = self.input_proj(x)
                residual1 = x
                
                # Attention
                x_seq = x.unsqueeze(1)
                attn_out, _ = self.attention(x_seq, x_seq, x_seq)
                x = residual1 + attn_out.squeeze(1)
                x = self.norm1(x)
                x = self.dropout(x)
                
                # FFN
                residual2 = x
                ffn_out = self.ffn(x)
                x = residual2 + ffn_out
                x = self.norm2(x)
                x = self.dropout(x)
                
                return self.output(x)
        
        return DynamicAttentionModel(
            len(self.feature_cols), 424, hidden_size, num_heads, ffn_multiplier, attn_dropout
        ).to(device)
    
    def _create_ensemble_model(self, num_experts, expert_size, gate_size, expert_dropout):
        """Создание EnsembleInsideModel с оптимизированными параметрами"""
        class DynamicEnsembleModel(nn.Module):
            def __init__(self, input_size, output_size, num_experts, expert_size, gate_size, expert_dropout):
                super().__init__()
                
                self.experts = nn.ModuleList([
                    nn.Sequential(
                        nn.Linear(input_size, expert_size),
                        nn.ReLU(),
                        nn.Dropout(expert_dropout),
                        nn.Linear(expert_size, expert_size // 2)
                    ) for _ in range(num_experts)
                ])
                
                self.gate = nn.Sequential(
                    nn.Linear(input_size, gate_size),
                    nn.ReLU(),
                    nn.Linear(gate_size, num_experts),
                    nn.Softmax(dim=1)
                )
                
                self.output = nn.Sequential(
                    nn.Linear(expert_size // 2, expert_size // 4),
                    nn.ReLU(),
                    nn.Dropout(expert_dropout * 0.7),
                    nn.Linear(expert_size // 4, output_size)
                )
            
            def forward(self, x):
                gates = self.gate(x)
                expert_outputs = [expert(x) for expert in self.experts]
                
                combined = torch.zeros_like(expert_outputs[0])
                for i, expert_out in enumerate(expert_outputs):
                    combined += gates[:, i:i+1] * expert_out
                
                return self.output(combined)
        
        return DynamicEnsembleModel(
            len(self.feature_cols), 424, num_experts, expert_size, gate_size, expert_dropout
        ).to(device)
    
    def _train_with_comprehensive_params(self, trial, model, optimizer, scheduler, criterion, 
                                       augmentation_strength, augmentation_methods, batch_size, grad_clip):
        """Обучение с комплексными параметрами"""
        
        # DataLoader с батчингом
        dataset = TensorDataset(self.X_train_t, self.y_train_t)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        best_val_score = -float('inf')
        patience = 20
        epochs_no_improve = 0
        best_epoch = 0
        
        for epoch in range(150):  # Увеличенное количество эпох
            # Training
            model.train()
            epoch_loss = 0
            num_batches = 0
            
            for batch_X, batch_y in dataloader:
                optimizer.zero_grad()
                
                # Аугментация с оптимизированными методами
                if augmentation_strength > 0 and augmentation_methods:
                    batch_X = advanced_augmentation(
                        batch_X, 
                        augmentation_methods=augmentation_methods,
                        augmentation_prob=augmentation_strength
                    )
                
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                optimizer.step()
                
                epoch_loss += loss.item()
                num_batches += 1
            
            # Validation
            model.eval()
            with torch.no_grad():
                val_outputs = model(self.X_val_t)
                val_score = calculate_spearman_score(
                    val_outputs.cpu().numpy(), 
                    self.y_val_t.cpu().numpy()
                )
            
            # Scheduler step
            if scheduler:
                if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                    scheduler.step(epoch_loss / num_batches)
                else:
                    scheduler.step()
            
            # Early stopping
            if val_score > best_val_score:
                best_val_score = val_score
                epochs_no_improve = 0
                best_epoch = epoch
                # Сохраняем лучшие веса
                best_weights = model.state_dict().copy()
            else:
                epochs_no_improve += 1
            
            # Report intermediate value for pruning
            trial.report(val_score, epoch)
            
            # Prune trial if not promising
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
            
            if epochs_no_improve >= patience:
                break
        
        # Загружаем лучшие веса
        if 'best_weights' in locals():
            model.load_state_dict(best_weights)
        
        # Очистка памяти
        del model
        torch.cuda.empty_cache()
        gc.collect()
        
        return best_val_score
    
    def optimize_comprehensive(self, n_trials=200, timeout=None):
        """Запуск комплексной оптимизации"""
        
        print("🚀 ЗАПУСК КОМПЛЕКСНОЙ OPTUNA ОПТИМИЗАЦИИ...")
        print("="*70)
        print(f"Количество trials: {n_trials}")
        print(f"Таймаут: {timeout} секунд")
        print("="*70)
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.HyperbandPruner()
        )
        
        study.optimize(
            self.objective_comprehensive, 
            n_trials=n_trials,
            timeout=timeout,
            gc_after_trial=True
        )
        
        self.studies['comprehensive'] = study
        self.best_params = study.best_params
        
        print("🎯 КОМПЛЕКСНАЯ ОПТИМИЗАЦИЯ ЗАВЕРШЕНА!")
        print(f"Лучший score: {study.best_value:.4f}")
        print(f"Лучшие параметры: {study.best_params}")
        
        return study

# Функция для создания лучшей модели на основе оптимизированных параметров
def create_best_comprehensive_model(optuna_optimizer):
    """Создает лучшую модель на основе комплексной оптимизации"""
    
    best_params = optuna_optimizer.best_params
    model_type = best_params['model_type']
    
    print(f"🎯 СОЗДАНИЕ ЛУЧШЕЙ МОДЕЛИ: {model_type}")
    
    if model_type == 'simple_effective':
        model = optuna_optimizer._create_simple_effective_model(
            best_params['hidden_size'],
            best_params['dropout_rate'],
            best_params['num_layers']
        )
        
    elif model_type == 'residual':
        model = optuna_optimizer._create_residual_model(
            best_params['res_hidden_size'],
            best_params['num_blocks'],
            best_params['block_dropout']
        )
        
    elif model_type == 'wide':
        model = optuna_optimizer._create_wide_model(
            best_params['layer1_size'],
            best_params['layer2_size'],
            best_params['layer3_size'],
            best_params['dropout1'],
            best_params['dropout2'],
            best_params['dropout3']
        )
        
    elif model_type == 'deep_skip':
        model = optuna_optimizer._create_deep_skip_model(
            best_params['deep_hidden_size'],
            best_params['deep_num_blocks'],
            best_params['deep_dropout']
        )
        
    elif model_type == 'attention':
        model = optuna_optimizer._create_attention_model(
            best_params['attn_hidden_size'],
            best_params['num_heads'],
            best_params['ffn_multiplier'],
            best_params['attn_dropout']
        )
        
    elif model_type == 'ensemble_inside':
        model = optuna_optimizer._create_ensemble_model(
            best_params['num_experts'],
            best_params['expert_size'],
            best_params['gate_size'],
            best_params['expert_dropout']
        )
    
    return model, best_params

# Функция для визуализации результатов Optuna
def visualize_optuna_results(studies):
    """Визуализация результатов Optuna"""
    
    for study_name, study in studies.items():
        print(f"\n📊 ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ: {study_name}")
        print("="*50)
        
        # История оптимизации
        try:
            fig = optuna.visualization.plot_optimization_history(study)
            fig.show()
        except Exception as e:
            print(f"⚠️ Не удалось создать историю оптимизации: {e}")
        
        # Важность параметров
        try:
            fig = optuna.visualization.plot_param_importances(study)
            fig.show()
        except Exception as e:
            print(f"⚠️ Не удалось создать график важности параметров: {e}")
        
        # Параллельные координаты
        try:
            fig = optuna.visualization.plot_parallel_coordinate(study)
            fig.show()
        except Exception as e:
            print(f"⚠️ Не удалось создать параллельные координаты: {e}")
        
        # Распределение параметров
        try:
            fig = optuna.visualization.plot_contour(study)
            fig.show()
        except Exception as e:
            print(f"⚠️ Не удалось создать контурный график: {e}")

# Обновленная функция инициализации с комплексной оптимизацией
def initialize_models_with_comprehensive_optuna(n_trials=200, timeout=None):
    global models, scaler, feature_cols, base_cols, model_val_scores, is_initialized, device
    
    if is_initialized:
        return
    
    print("="*70)
    print(f"КОМПЛЕКСНАЯ OPTUNA ОПТИМИЗАЦИЯ НА {device}")
    print("="*70)
    
    # Загрузка данных
    train = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/train.csv')
    train_labels = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/train_labels.csv')
    
    # Базовые колонки
    base_cols = [c for c in train.columns 
                 if c not in ['date_id'] 
                 and pd.api.types.is_numeric_dtype(train[c])]
    
    print(f"Базовых колонок: {len(base_cols)}")
    print("Создание оптимизированных признаков...")
    
    # Создание признаков
    train_features = create_optimized_features(train, base_cols_ref=base_cols, max_features=800)
    
    # Выбор лучших признаков
    target_cols = [f'target_{i}' for i in range(424)]
    selected_features = select_best_features_vectorized_fixed(train_features, train_labels[target_cols], n_features=500)
    
    feature_cols = selected_features
    
    # Подготовка данных
    X_train = train_features[feature_cols].fillna(0).values
    y_train = train_labels[target_cols].fillna(0).values
    
    X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
    y_train = np.nan_to_num(y_train, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Масштабирование
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Перенос на GPU
    X_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_tensor = torch.FloatTensor(y_train).to(device)
    
    # Разделение на train/validation
    split_idx = int(len(X_train_scaled) * 0.85)
    X_train_t, X_val_t = X_tensor[:split_idx], X_tensor[split_idx:]
    y_train_t, y_val_t = y_tensor[:split_idx], y_tensor[split_idx:]
    
    print(f"Train: {len(X_train_t)}, Validation: {len(X_val_t)}")
    
    # Комплексная оптимизация Optuna
    optuna_optimizer = ComprehensiveOptunaOptimizer(
        X_train_t, y_train_t, X_val_t, y_val_t, feature_cols
    )
    
    study = optuna_optimizer.optimize_comprehensive(
        n_trials=n_trials, 
        timeout=timeout
    )
    
    # Создание и обучение лучшей модели
    best_model, best_params = create_best_comprehensive_model(optuna_optimizer)
    
    # Извлекаем оптимизированные параметры для обучения
    lr = best_params['lr']
    augmentation_strength = best_params['augmentation_strength']
    
    # Конфигурация аугментации
    augmentation_methods = []
    if best_params.get('use_noise', False):
        augmentation_methods.append(('noise', best_params.get('noise_prob', 0.3)))
    if best_params.get('use_scaling', False):
        augmentation_methods.append(('scaling', best_params.get('scaling_prob', 0.2)))
    if best_params.get('use_jitter', False):
        augmentation_methods.append(('jitter', best_params.get('jitter_prob', 0.2)))
    if best_params.get('use_dropout_aug', False):
        augmentation_methods.append(('dropout', best_params.get('dropout_aug_prob', 0.1)))
    if best_params.get('use_permutation', False):
        augmentation_methods.append(('permutation', best_params.get('permutation_prob', 0.1)))
    
    print(f"\n🎯 ОБУЧЕНИЕ ЛУЧШЕЙ МОДЕЛИ С ОПТИМИЗИРОВАННЫМИ ПАРАМЕТРАМИ:")
    print(f"   Архитектура: {best_params['model_type']}")
    print(f"   Learning rate: {lr}")
    print(f"   Augmentation strength: {augmentation_strength}")
    print(f"   Augmentation methods: {[m[0] for m in augmentation_methods]}")
    
    # Обучение лучшей модели
    best_model, val_score = train_model_with_augmentation(
        best_model, X_train_t, y_train_t, X_val_t, y_val_t,
        epochs=300,  # Увеличиваем эпохи для лучшей модели
        patience=25,
        lr=lr,
        model_name=f"BEST_{best_params['model_type']}",
        use_augmentation=True,
        augmentation_strength=augmentation_strength
    )
    
    models.append(best_model)
    model_val_scores.append(val_score)
    
    print(f"✅ ЛУЧШАЯ МОДЕЛЬ завершена. Best Val Score: {val_score:.4f}")
    
    # Визуализация результатов
    visualize_optuna_results(optuna_optimizer.studies)
    
    is_initialized = True
    
    # Сохранение scaler и feature_cols
    joblib.dump(scaler, 'scaler_optimized.joblib')
    joblib.dump(feature_cols, 'feature_cols_optimized.joblib')
    joblib.dump(best_params, 'best_params.joblib')
    print("Scaler, feature_cols и best_params сохранены")

# Новая main функция с комплексной оптимизацией
def main_with_comprehensive_optuna(n_trials=200, timeout=None):
    global is_initialized
    
    print("🚀 ЗАПУСК ПАЙПЛАЙНА С КОМПЛЕКСНОЙ OPTUNA ОПТИМИЗАЦИЕЙ")
    print("="*70)
    print(f"Количество trials: {n_trials}")
    print(f"Таймаут: {timeout} секунд")
    print("="*70)
    
    start_time = time.time()
    
    if not is_initialized:
        print("Начало комплексной оптимизации...")
        initialize_models_with_comprehensive_optuna(
            n_trials=n_trials, 
            timeout=timeout
        )
    
    # Оценка
    print("\n" + "="*70)
    print("ОЦЕНКА ОПТИМИЗИРОВАННОЙ МОДЕЛИ")
    print("="*70)
    
    results = full_evaluation()
    
    # Создание submission
    print("\nСоздание submission файла...")
    create_submission_file()
    
    end_time = time.time()
    total_time = (end_time - start_time) / 3600  # в часах
    
    print("\n✅ КОМПЛЕКСНАЯ OPTUNA ОПТИМИЗАЦИЯ ЗАВЕРШЕНА!")
    print(f"🏆 Ваш ожидаемый Kaggle Score: {results['validation']['kaggle_score']:.2f}")
    print(f"📊 Sharpe Ratio: {results['validation']['sharpe_ratio']:.6f}")
    print(f"🔢 Количество моделей: {len(models)}")
    print(f"⏱️ Общее время выполнения: {total_time:.2f} часов")
    print(f"🎯 Использована комплексная Optuna оптимизация")
    print(f"📁 Submission файл: submission.parquet")
    print(f"\n🚀 ОТПРАВЛЯЙТЕ НА KAGGLE!")

In [22]:
def advanced_augmentation(features, augmentation_methods=None, augmentation_prob=0.7):
    """Продвинутая аугментация с кастомными методами"""
    if augmentation_methods is None:
        augmentation_methods = [
            ('noise', 0.3),
            ('scaling', 0.2), 
            ('jitter', 0.2),
            ('dropout', 0.2),
            ('permutation', 0.1)
        ]
    
    augmented_features = features.clone()
    
    # Применяем аугментации с вероятностью
    if torch.rand(1).item() < augmentation_prob:
        for method, prob in augmentation_methods:
            if torch.rand(1).item() < prob:
                if method == 'noise':
                    augmented_features = TimeSeriesAugmentation.add_gaussian_noise(augmented_features, 0.01)
                elif method == 'scaling':
                    augmented_features = TimeSeriesAugmentation.scaling_augmentation(augmented_features)
                elif method == 'jitter':
                    augmented_features = TimeSeriesAugmentation.jittering(augmented_features)
                elif method == 'dropout':
                    augmented_features = TimeSeriesAugmentation.feature_dropout(augmented_features, 0.05)
                elif method == 'permutation':
                    augmented_features = TimeSeriesAugmentation.random_permutation(augmented_features, 0.05)
    
    return augmented_features

In [23]:
# Добавьте этот импорт в начало
import time

# Запуск комплексной оптимизации
if __name__ == "__main__":
    # Для серьезной оптимизации (рекомендуется если есть время)
    # n_trials=200-500, timeout=24-48 часов
    
    main_with_comprehensive_optuna(
        n_trials=500,           # Количество trials
        timeout=48*3600         # 48 часов таймаут
    )

🚀 ЗАПУСК ПАЙПЛАЙНА С КОМПЛЕКСНОЙ OPTUNA ОПТИМИЗАЦИЕЙ
Количество trials: 500
Таймаут: 172800 секунд

ОЦЕНКА ОПТИМИЗИРОВАННОЙ МОДЕЛИ

🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
ПОЛНАЯ ОЦЕНКА МОДЕЛИ
🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
Обрабатывается 100 базовых колонок...
Достигнут лимит в 800 признаков
Создано 1507 признаков после фильтрации

1️⃣  VALIDATION SCORE:
Validation set size: 295 samples

ВЫЧИСЛЕНИЕ KAGGLE SCORE (Modified Sharpe Ratio)

Статистика Spearman Correlations:
  Успешно: 424/424 targets
  Mean:    0.057219
  Std:     0.058726
  Min:     -0.118412
  Max:     0.232985

  Positive correlations: 359/424 (84.7%)
  Negative correlations: 65/424 (15.3%)

KAGGLE SCORE (Modified Sharpe Ratio):
  Sharpe Ratio:        0.974347
  KAGGLE SCORE:        97434.67

  ✓ Хороший результат (> 50,000)


2️⃣  TEST SCORE:
Обрабатывается 100 базовых колонок...
Достигнут лимит в 800 признаков
Создано 1502 признаков после фильтрации

ВЫЧИСЛЕНИЕ KAGGLE SCORE (Modified Sharpe Ratio)

Статистика Spearma